In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import joblib
from sklearn.decomposition import PCA

In [2]:
data = pd.read_csv('../data/train_test_data/test.csv')
data.head()

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,is_canceled
0,Resort Hotel,238,2016,September,37,7,0,3,1,0.0,...,1.0,NaN,0,Transient,89.00,0,0,Canceled,2016-02-09,1
1,Resort Hotel,33,2017,June,25,19,1,1,2,0.0,...,240.0,NaN,0,Transient,154.00,1,1,Check-Out,2017-06-21,0
2,City Hotel,239,2015,October,42,17,0,1,2,0.0,...,1.0,NaN,0,Transient,60.00,0,0,Canceled,2015-08-17,1
3,Resort Hotel,64,2015,September,39,24,0,3,2,0.0,...,240.0,NaN,0,Transient,80.10,0,2,Check-Out,2015-09-27,0
4,City Hotel,34,2016,October,41,2,2,3,2,0.0,...,7.0,NaN,0,Transient,95.47,0,1,Check-Out,2016-10-07,0


In [3]:
print("Rows :  ", data.shape[0])
print("Columns:  ", data.shape[1])
print()
print("Feature types:  ")
print(data.dtypes)
print()
print('Missing values : ')
print(data.isnull().sum())
print()
print('Missing value rate : ')
print(data.isnull().sum()/data.shape[0])
print()
print('Unique values:')
print(data.nunique())

Rows :   23878
Columns:   32

Feature types:  
hotel                              object
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                        

In [4]:
# create new feature: 'reserved=assigned'
def function(a, b):
    if a == b:
        return "Yes"
    else:
        return "No"
data['reserved=assigned'] = data.apply(lambda x : function(x['assigned_room_type'],x['reserved_room_type']),axis = 1)

In [5]:
# create new feature: 'day_of_week'
data['reservation_status_date'] = pd.to_datetime(data['reservation_status_date'], errors='coerce')
data['day_of_week'] = data['reservation_status_date'].dt.dayofweek
transform_num_day_of_week = {0: 'Monday',
                             1: 'Tuesday',
                             2: 'Wednesday',
                             3: 'Thursday',
                             4: 'Friday',
                             5: 'Saturday',
                             6: 'Sunday'}
data['day_of_week'] = data['day_of_week'].map(transform_num_day_of_week)

transform_day_weekday = {'Monday': 'Yes',
                         'Tuesday': 'Yes',
                         'Wednesday': 'Yes',
                         'Thursday': 'Yes',
                         'Friday': 'Yes',
                         'Saturday': 'No',
                         'Sunday': 'No'}
data['weekday?'] = data['day_of_week'].map(transform_day_weekday)

In [6]:
col_del=[]
to_num=[]
to_cat=[]

In [7]:
to_cat.append('adults')
to_cat.append('children')
to_cat.append('babies')
to_cat.append('is_repeated_guest')
to_cat.append('previous_cancellations')
to_cat.append('previous_bookings_not_canceled')
to_cat.append('booking_changes')
to_cat.append('days_in_waiting_list')
to_cat.append('total_of_special_requests')
to_cat.append('required_car_parking_spaces')
to_cat.append('is_canceled')
to_cat.append('stays_in_week_nights')
to_cat.append('stays_in_weekend_nights')

In [8]:
col_del.append('reservation_status_date')
col_del.append('reserved_room_type')
col_del.append('reservation_status')
col_del.append('day_of_week')
col_del.append('agent')
col_del.append('arrival_date_year')
col_del.append('arrival_date_week_number')
col_del.append('arrival_date_day_of_month')
data.drop(col_del,axis=1,inplace=True)

In [9]:
index1 = data.stays_in_week_nights>5
index2 = data.stays_in_weekend_nights>2

data.loc[index1,'stays_in_week_nights']=100000
data.loc[index2,'stays_in_weekend_nights']=100000

In [10]:
# drop features with too high missing rate
for col in data.columns:
    if data[col].isnull().sum()/data.shape[0] >= 0.8:
        data.drop(col,axis=1,inplace=True)

In [11]:
data.dtypes

hotel                              object
lead_time                           int64
arrival_date_month                 object
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
days_in_waiting_list                int64
customer_type                      object
adr                               float64
required_car_parking_spaces         int64
total_of_special_requests           int64
is_canceled                       

In [12]:
# to categorical
for col in to_cat:
    data[col] = data[col].astype(object)

In [13]:
data.dtypes

hotel                              object
lead_time                           int64
arrival_date_month                 object
stays_in_weekend_nights            object
stays_in_week_nights               object
adults                             object
children                           object
babies                             object
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                  object
previous_cancellations             object
previous_bookings_not_canceled     object
assigned_room_type                 object
booking_changes                    object
deposit_type                       object
days_in_waiting_list               object
customer_type                      object
adr                               float64
required_car_parking_spaces        object
total_of_special_requests          object
is_canceled                       

In [14]:
cat_cols=[]
num_cols=[]

# select_col_num_features
for col_name in data.columns:
    if data[col_name].dtypes == 'object':
        cat_cols.append(col_name)
    else:
        num_cols.append(col_name)

print("categorical features: ")
print(cat_cols)
print("numerical features: ")
print(num_cols)

categorical features: 
['hotel', 'arrival_date_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'meal', 'country', 'market_segment', 'distribution_channel', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'assigned_room_type', 'booking_changes', 'deposit_type', 'days_in_waiting_list', 'customer_type', 'required_car_parking_spaces', 'total_of_special_requests', 'is_canceled', 'reserved=assigned', 'weekday?']
numerical features: 
['lead_time', 'adr']


In [15]:
# impute categorial features with low missing rate
for col in cat_cols:
    if data[col].isnull().sum()/data.shape[0] <= 0.2:
        freq = data[col].value_counts().idxmax()
        data[col].fillna(value=freq, inplace=True)

In [16]:
data.isnull().sum()

hotel                             0
lead_time                         0
arrival_date_month                0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests         0
is_canceled                       0
reserved=assigned                 0
weekday?                          0
dtype: int64

In [17]:
# mutil->bin for categorical features
to_cat.remove('stays_in_week_nights')
to_cat.remove('stays_in_weekend_nights')
for col in to_cat:
    data[col] = ['0' if x == 0 else '>0' for x in data[col]]

In [18]:
print("categorical features: ")
print(cat_cols)
print("numerical features: ")
print(num_cols)

categorical features: 
['hotel', 'arrival_date_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'meal', 'country', 'market_segment', 'distribution_channel', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'assigned_room_type', 'booking_changes', 'deposit_type', 'days_in_waiting_list', 'customer_type', 'required_car_parking_spaces', 'total_of_special_requests', 'is_canceled', 'reserved=assigned', 'weekday?']
numerical features: 
['lead_time', 'adr']


In [19]:
data.nunique()

hotel                                2
lead_time                          462
arrival_date_month                  12
stays_in_weekend_nights              4
stays_in_week_nights                 7
adults                               2
children                             2
babies                               2
meal                                 5
country                            135
market_segment                       7
distribution_channel                 5
is_repeated_guest                    2
previous_cancellations               2
previous_bookings_not_canceled       2
assigned_room_type                  11
booking_changes                      2
deposit_type                         3
days_in_waiting_list                 2
customer_type                        4
adr                               3777
required_car_parking_spaces          2
total_of_special_requests            2
is_canceled                          2
reserved=assigned                    2
weekday?                 

In [20]:
# merge rare categories to other
to_merge_rare = []
to_merge_rare.append('assigned_room_type')
to_merge_rare.append('country')
to_merge_rare.append('market_segment')

for col in to_merge_rare:
    print(col)
    series = pd.value_counts(data[col])
    mask = (series / series.sum() * 100).lt(5)  # 10 is the percentage
    data = data.assign(temp=np.where(data[col].isin(
        series[mask].index), 'Other', data[col]))
    data[col] = data.temp
    data.drop('temp', axis=1, inplace=True)
    print(data[col].value_counts())

assigned_room_type
A        14750
D         5047
Other     2528
E         1553
Name: assigned_room_type, dtype: int64
country
PRT      9825
Other    6294
GBR      2415
FRA      2086
ESP      1770
DEU      1488
Name: country, dtype: int64
market_segment
Online TA        11241
Offline TA/TO     4847
Groups            3981
Direct            2535
Other             1274
Name: market_segment, dtype: int64


In [21]:
data.nunique()

hotel                                2
lead_time                          462
arrival_date_month                  12
stays_in_weekend_nights              4
stays_in_week_nights                 7
adults                               2
children                             2
babies                               2
meal                                 5
country                              6
market_segment                       5
distribution_channel                 5
is_repeated_guest                    2
previous_cancellations               2
previous_bookings_not_canceled       2
assigned_room_type                   4
booking_changes                      2
deposit_type                         3
days_in_waiting_list                 2
customer_type                        4
adr                               3777
required_car_parking_spaces          2
total_of_special_requests            2
is_canceled                          2
reserved=assigned                    2
weekday?                 

In [22]:
def find_outliers_tukey(feature):
    q1 = np.percentile(feature, 25)
    q3 = np.percentile(feature, 75)
    iqr = q3-q1
    upbound = q3+1.5*iqr
    downbound = q1 - 1.5*iqr
    outlier_indices = list(feature.index[(feature>upbound)|(feature<downbound)])
    outlier_values = list(feature[outlier_indices])
    return outlier_indices,outlier_values

In [23]:
# find outlier by 1.5IQR, then impute with median
for col in num_cols:
    outlier_indices,outlier_values = find_outliers_tukey(data[col])
    data.loc[outlier_indices,col]=data[col].median()

In [24]:
num_cols_skew=[]
for col in num_cols:
    print("Skewness for "+str(col)+": " +str(data[col].skew()))
    if data[col].skew()>1 or data[col].skew()<-1:
        data[col] = np.log1p(data[col])
        print("Skewness for "+str(col)+": " +str(data[col].skew()))

Skewness for lead_time: 1.0302882811908833
Skewness for lead_time: -0.9119716155356801
Skewness for adr: 0.3782843094026922


In [25]:
# find 2-class and multi-class features
bin_cols = data.nunique()[data.nunique()==2].keys().tolist()
multi_cols = [col for col in cat_cols if col not in bin_cols]

In [26]:
print("2-class features: ")
print(bin_cols)
print("multi-class features: ")
print(multi_cols)

2-class features: 
['hotel', 'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'days_in_waiting_list', 'required_car_parking_spaces', 'total_of_special_requests', 'is_canceled', 'reserved=assigned', 'weekday?']
multi-class features: 
['arrival_date_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'meal', 'country', 'market_segment', 'distribution_channel', 'assigned_room_type', 'deposit_type', 'customer_type']


In [27]:
# encode categorical features
le = LabelEncoder()
data_bin = data[bin_cols]
for col in bin_cols:
    data_bin[col] = le.fit_transform(data_bin[col])
    
data_multi = pd.get_dummies(data = data[multi_cols], columns=multi_cols)

# scaling numerical features
import joblib
ss = joblib.load("../models/scaler.m")
data_num = ss.transform(data[num_cols])  # no fit here !!!
data_num = pd.DataFrame(data_num,columns=num_cols)

/Users/FY/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [28]:
new_data = pd.concat([data_bin,data_multi,data_num],axis=1)
new_data.head()

,hotel,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,required_car_parking_spaces,...,assigned_room_type_Other,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,lead_time,adr
0,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1.071102,-0.202054
1,1,1,0,0,0,0,0,0,0,1,...,1,1,0,0,0,0,1,0,-0.171189,1.429752
2,0,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,1.073762,-0.930091
3,1,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0.241629,-0.425486
4,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,-0.152723,-0.039627


In [29]:
new_data.shape

(23878, 71)

In [30]:
train_data = pd.read_csv('../data/featured_data/featured_train.csv')
train_col=train_data.columns.to_list()
test_col=new_data.columns.to_list()
print(train_col==test_col)  # same features in training set and test set!!!

True


In [31]:
new_data.to_csv('../data/featured_data/featured_test.csv',index=False)

In [32]:
X = new_data.drop('is_canceled',axis=1)
y = new_data.is_canceled

In [33]:
pca = joblib.load("../models/pca.m")
X_pca = pca.transform(X)  # not fit here!!!

In [34]:
pd.DataFrame(X_pca).to_csv('../data/featured_data/X_test.csv',index=False)
pd.DataFrame(y).to_csv('../data/featured_data/y_test.csv',index=False)

In [35]:
X_pca.shape

(23878, 48)